In [10]:
# Load LSTM network and generate text
import sys
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import h5py
import re
import pandas as pd
from pprint import pprint

In [38]:
mov_lines_file=open("/home/kavin/Silo/College Work/AI/DataSets/cornell_movie_dialogs_corpus/cornell movie-dialogs corpus/movie_lines.txt",'r')

In [39]:
mov_lines=mov_lines_file.readlines()

In [91]:
dialogues=[]
for mov_line in mov_lines:
    matched=re.match('.* \+\+\+\$\+\+\+ (.+)', mov_line)
    dialogues.append(matched.group(1))

In [232]:
dialogues[1]

'They do to!'

In [239]:
tokenized_dialogues=[]
for line in dialogues:
    tokenized_dialogues.append(filter(None,re.split(' |!|\.|:|;|,|\?|"|\-\-|\*|<.*?>|</.*?>', line.lower())))

In [240]:
tokenized_dialogues[0]

['they', 'do', 'not']

In [241]:
cleaned_dialogues=[]
for line in tokenized_dialogues:
    line_list=[]
    for token in line:
        try:
            matched=re.match('([\*|\'|_|\-|\[]|)(.*)([\*|\'|_|\-|\]]|)', token)
            line_list.append(matched.group(2))
        except:
            pass
    cleaned_dialogues.append(line_list)

In [245]:
cleaned_dialogues[0]

['they', 'do', 'not']

In [247]:
all_dialogues=[]
for line in cleaned_dialogues:
    all_dialogues+=line
tokens=pd.Series(all_dialogues).unique()

In [248]:
bag_of_words={}
index_to_words={}
index=0
for word in tokens:
    bag_of_words[word]=index
    index_to_words[index]=word
    index+=1
n_chars = len(tokenized)
n_vocab = len(tokens)

In [249]:
n_vocab

62141

In [250]:
cleaned_dialogues[:10]

[['they', 'do', 'not'],
 ['they', 'do', 'to'],
 ['i', 'hope', 'so'],
 ['she', 'okay'],
 ["let's", 'go'],
 ['wow'],
 ['okay', "you're", 'gonna', 'need', 'to', 'learn', 'how', 'to', 'lie'],
 ['no'],
 ["i'm",
  'kidding',
  'you',
  'know',
  'how',
  'sometimes',
  'you',
  'just',
  'become',
  'this',
  'persona',
  'and',
  'you',
  "don't",
  'know',
  'how',
  'to',
  'quit'],
 ['like', 'my', 'fear', 'of', 'wearing', 'pastels']]

In [251]:
indexed_dialogues=[]
for line in cleaned_dialogues:
    line_list=[]
    for token in line:
        try:
            line_list.append(bag_of_words[token])
        except:
            line_list.append('-1')
    indexed_dialogues.append(line_list)

In [252]:
indexed_dialogues[:10]

[[0, 1, 2],
 [0, 1, 3],
 [4, 5, 6],
 [7, 8],
 [9, 10],
 [11],
 [8, 12, 13, 14, 3, 15, 16, 3, 17],
 [18],
 [19, 20, 21, 22, 16, 23, 21, 24, 25, 26, 27, 28, 21, 29, 22, 16, 3, 30],
 [31, 32, 33, 34, 35, 36]]

In [3]:
tokens=pd.read_csv("/home/kavin/Silo/College Work/DS501/Case 5/10000_email_tokens.csv", names=("tokens",))


In [33]:
cursors_list=collection.find({"x_folder":{"$regex":".*INBOX.*","$options": "-i"}}, {"subject":1, "_id":0})

In [34]:
all_subjects=[]
unique_subjects=[]
for cursor in cursors_list[:10000]:
    all_subjects.append(re.sub('.*FW:.*|.*RE:.*|.*fw:.*|.*re:.*|.*Fw:.*|.*Re:.*', '', cursor['subject']))

In [47]:
unique_subjects=pd.Series(all_subjects).unique()

In [48]:
all_rec_msgs=[]
all_sent_msgs=[]
for subject in unique_subjects[:10]:
    cursors_list=collection.find({"x_folder":{"$regex":".*INBOX.*","$options": "-i"}, "subject":subject}, {"message":1, "_id":0})
    for cursor in cursors_list:
        all_rec_msgs.append(cursor['message'])
    cursors_list=collection.find({"x_folder":{"$regex":".*sent.*","$options": "-i"}, "subject":subject}, {"message":1, "_id":0})
    for cursor in cursors_list:
        all_sent_msgs.append(cursor['message'])
    

In [49]:
sent_tokenized=[]
for message in all_sent_msgs:
    sent_tokenized+=re.split('\W+|,| |\*| \n|\t|:|_', message)
recv_tokenized=[]
for message in all_rec_msgs:
    recv_tokenized+=re.split('\W+|,| |\*| \n|\t|:|_', message)


In [50]:
tokens_file=open("/home/kavin/Silo/College Work/DS501/Case 5/sent_email_tokens.csv",'w')
for token in sent_tokenized:
    if len(token) >0:
        tokens_file.write(token+"\n")
tokens_file.close()
tokens_file=open("/home/kavin/Silo/College Work/DS501/Case 5/received_email_tokens.csv",'w')
for token in recv_tokenized:
    if len(token) >0:
        tokens_file.write(token+"\n")
tokens_file.close()

In [55]:
sent_tokens=pd.read_csv("/home/kavin/Silo/College Work/DS501/Case 5/sent_email_tokens.csv", names=("tokens",))
recv_tokens=pd.read_csv("/home/kavin/Silo/College Work/DS501/Case 5/received_email_tokens.csv", names=("tokens",))

In [ ]:
tokens=sent_tokens["tokens"].append(recv_tokens["tokens"])
tokens

In [61]:
unique_words=pd.Series(tokens).unique()
bag_of_words={}
index_to_words={}
index=0.0
for word in unique_words:
    bag_of_words[word]=index
    index_to_words[index]=word
    index+=1
n_chars = len(tokens)
n_vocab = len(unique_words)

In [62]:
n_chars

17285

In [66]:
tokens[112]

112       To
112    being
Name: tokens, dtype: object

In [ ]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = tokens[i:i + seq_length]["tokens"]
	seq_out = tokens["tokens"][i + seq_length]
	dataX.append([bag_of_words[word] for word in seq_in])
	dataY.append(bag_of_words[seq_out])
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns

In [7]:
len(dataX)

499900

In [16]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [17]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=5, batch_size=128, callbacks=callbacks_list)

Epoch 1/5
99900/99900 [==============================] - 442s - loss: 7.7469   
Epoch 2/5
89344/99900 [=========================>....] - ETA: 45s - loss: 7.5464

In [6]:
# load the network weights
filename = "weights-improvement-13-1.7987.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print "Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""

In [ ]:
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print "\nDone."